<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E9%99%84%E9%8C%84_%E7%B4%85%E6%A8%93%E5%A4%A2%E7%94%9F%E6%88%90%E5%99%A8%E7%9A%84%E5%8E%9F%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense, Embedding
from tensorflow.keras.utils import to_categorical

In [4]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream.txt \
    -O /content/dream.txt

--2023-02-19 22:48:36--  https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2656053 (2.5M) [text/plain]
Saving to: ‘/content/dream.txt’

/content/dream.txt  100%[===================>]   2.53M  --.-KB/s    in 0.08s   

2023-02-19 22:48:37 (32.0 MB/s) - ‘/content/dream.txt’ saved [2656053/2656053]



In [8]:
f = open('/content/dream.txt', 'r')
lines = f.readlines()
f.close()

In [9]:
text_lines = [x.lstrip('\u3000\u3000') for x in lines]

In [10]:
text = ''.join(text_lines)

你當然可以欣賞一下裡面內容，不過太長了了這裡省略。

In [11]:
# text

### 訓練 tokenizer

In [12]:
tokenizer = Tokenizer(char_level=True)

In [13]:
tokenizer.fit_on_texts([text])

試用一下

In [14]:
tokenizer.texts_to_sequences(["使人一見便知是奇物方妙。"])

[[250, 12, 6, 32, 37, 54, 13, 527, 370, 130, 440, 2]]

變成文字。

In [15]:
''.join(tokenizer.sequences_to_texts([[80, 113, 113, 139, 445, 39]])[0].split())

'今天天氣很好'

計算最大 id, 即不同的字的數目。

In [16]:
max_id = len(tokenizer.word_index)

In [17]:
[encoded] = np.array(tokenizer.texts_to_sequences([text]))-1

In [18]:
dataset = tf.data.Dataset.from_tensor_slices(encoded)

In [19]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [20]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [21]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:,1:]))

In [22]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [23]:
dataset = dataset.prefetch(1)

In [24]:
model = Sequential()

In [25]:
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=[None, max_id]))

In [26]:
model.add(TimeDistributed(Dense(max_id, activation="softmax")))

In [27]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 128)         2381312   
                                                                 
 time_distributed (TimeDistr  (None, None, 4522)       583338    
 ibuted)                                                         
                                                                 
Total params: 2,964,650
Trainable params: 2,964,650
Non-trainable params: 0
_________________________________________________________________


開始訓練。

In [ ]:
history = model.fit(dataset, epochs=10)

Epoch 1/10
27545/27545 [==============================] - 3848s 140ms/step - loss: 4.2496
Epoch 2/10
27545/27545 [==============================] - 3846s 140ms/step - loss: 3.1537
Epoch 3/10
27545/27545 [==============================] - 3846s 140ms/step - loss: 2.9128
Epoch 4/10
27545/27545 [==============================] - 3842s 139ms/step - loss: 2.8136
Epoch 5/10
27545/27545 [==============================] - 3842s 139ms/step - loss: 2.7610
Epoch 6/10
27545/27545 [==============================] - 3843s 139ms/step - loss: 2.7279
Epoch 7/10
27545/27545 [==============================] - 3842s 139ms/step - loss: 2.7051
Epoch 8/10
27545/27545 [==============================] - 3841s 139ms/step - loss: 2.6880
Epoch 9/10
27545/27545 [==============================] - 3842s 139ms/step - loss: 2.6755
Epoch 10/10
27545/27545 [==============================] - 3841s 139ms/step - loss: 2.6661


#### 把 model 存起來

In [ ]:
model.save("dream_rnn")

INFO:tensorflow:Assets written to: dream_rnn/assets


#### 把 tokenizer 存起來 

In [ ]:
import pickle

In [ ]:
f = open('dream_tokenizer.pkl', 'wb')
pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)
f.close()